## <font color=green>This is to monitor database Locks,performance metics</font>

This notebook will help the dba's to quickly gte some performance metics for db2 luw database.

If you find any new sql statements please add notes and share.

Step1:
Import the required **Python libraries**

In [1]:
import sys,os,os.path
os.environ['IBM_DB_HOME']='C:\Program Files\IBM\SQLLIB'
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
from IPython.display import HTML
#Import stuff we need
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import numpy as np
import pandas as pd

# All plots should be inline, inside the notebook
%matplotlib inline


In [2]:
## Hide code cells
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

### <font color=green>Connect to the database</font>

In [3]:
%sql db2+ibm_db://db2inst1:UECtZ3x9@db2mbruat1:60000/ods

'Connected: db2inst1@ods'

## <font color=green>Current a database status on all partitions/Members</font> 


### <font color=green>Database active since</font> 


In [4]:
db_conn_time=%sql select member,db_conn_time as Database_activated from table(mon_get_database(-2)) as mgd
db_conn_time

 * db2+ibm_db://db2inst1:***@db2mbruat1:60000/ods
Done.


MEMBER,database_activated
2,2018-03-29 12:31:47
1,2018-04-26 12:02:30
0,2018-03-29 10:52:05


###  <font color=green> Checks the status of the database on all members
#### <font color=green>If the VAULE AND Deferred_value are same then we are good</font> 

In [5]:
%sql select distinct varchar(NAME,20)  as CONFIGURATION_PARAMETER,Member as member ,substr(VALUE,1,45) as VALUE,substr(DEFERRED_VALUE,1,45) as deferred_value \
FROM SYSIBMADM.DBCFG  \
where name in ('database_consistent','backup_pending','restore_pending','rollfwd_pending')  \
order by 1,2,3 with ur


 * db2+ibm_db://db2inst1:***@db2mbruat1:60000/ods
Done.


configuration_parameter,MEMBER,VALUE,deferred_value
backup_pending,0,NO,NO
backup_pending,1,NO,NO
backup_pending,2,NO,NO
database_consistent,0,NO,NO
database_consistent,1,NO,NO
database_consistent,2,NO,NO
restore_pending,0,NO,NO
restore_pending,1,NO,NO
restore_pending,2,NO,NO
rollfwd_pending,0,NO,NO


### <font color=green>List of all the database paths *includes db/storage/log paths*</font> 

In [6]:
%sql  SELECT DBPARTITIONNUM as DBPART,SUBSTR(TYPE,1,30) AS DB_PATH_TYPE, SUBSTR(PATH,1,80) AS PATH_NAME FROM SYSIBMADM.DBPATHS ORDER BY 2,1 with ur;


 * db2+ibm_db://db2inst1:***@db2mbruat1:60000/ods
Done.


dbpart,db_path_type,path_name
0,DBPATH,/udata/ODS/METADATA/db2inst1/NODE0000/SQL00001/
0,DBPATH,/udata/ODS/METADATA/db2inst1/NODE0000/SQL00001/MEMBER0000/
0,DBPATH,/udata/ODS/METADATA/db2inst1/NODE0000/SQL00001/MEMBER0001/
0,DBPATH,/udata/ODS/METADATA/db2inst1/NODE0000/SQL00001/MEMBER0002/
0,DB_STORAGE_PATH,/udata/ODS/PROINDX/
0,DB_STORAGE_PATH,/udata/ODS/PRODATA/
0,DB_STORAGE_PATH,/udata/ODS/HISTIDX/
0,DB_STORAGE_PATH,/udata/ODS/IDX/
0,DB_STORAGE_PATH,/udata/ODS/TEMP/
0,DB_STORAGE_PATH,/odsdt/ODS/DATA/


### <font color=green>Last sucessfull database backup</font>  

In [7]:
a=%sql select max(substr(START_TIME,1,8)) as date from SYSIBMADM.DB_HISTORY where operation in ('B') and SQLCODE is null with ur
a

 * db2+ibm_db://db2inst1:***@db2mbruat1:60000/ods
Done.


DATE
20180319


# <font color=green>Health checks for the database</font>  

## **Table space** state for this database
If the tablespace is not in ***NORMAL*** state then we have a problem

In [8]:
%sql select substr(tbsp_name,1,20) as tbsp, TBSP_TYPE, member as dbpart, substr(tbsp_state,1,30) as state from TABLE(MON_GET_TABLESPACE('',-2)) AS t  \
order by tbsp, dbpart with ur ;

 * db2+ibm_db://db2inst1:***@db2mbruat1:60000/ods
Done.


tbsp,tbsp_type,dbpart,state
EXPLAIN_TBSP,DMS,0,NORMAL
EXPLAIN_TBSP,DMS,1,NORMAL
EXPLAIN_TBSP,DMS,2,NORMAL
QCASN,DMS,0,NORMAL
QCASN,DMS,1,NORMAL
QCASN,DMS,2,NORMAL
QCASN1,DMS,0,NORMAL
QCASN1,DMS,1,NORMAL
QCASN1,DMS,2,NORMAL
QCASN12,DMS,0,NORMAL


<font color=green>This would give you the status of the tablespace in ***abnormal*** state</font> 



In [9]:
%sql select substr(tbsp_name,1,20) as tbsp, TBSP_TYPE, member as dbpart, substr(tbsp_state,1,30) as state from TABLE(MON_GET_TABLESPACE('',-2)) AS t  where TBSP_STATE != 'NORMAL' \
order by tbsp, dbpart with ur

 * db2+ibm_db://db2inst1:***@db2mbruat1:60000/ods
Done.


tbsp,tbsp_type,dbpart,state


##  <font color=green> Container status IF Accessible is 1 then the container is in normal state</font>

In [10]:
%sql select member as dbpart , substr(tbsp_name,1, 18) as tbspname , \
substr(container_name,1, 75) as container_name, \
ACCESSIBLE \
from TABLE(MON_GET_CONTAINER('',-2)) AS t 


 * db2+ibm_db://db2inst1:***@db2mbruat1:60000/ods
Done.


dbpart,tbspname,container_name,accessible
1,SYSCATSPACE,/udata/ODS/IBMTBSP/db2inst1/NODE0000/ODS/T0000000/C0000000.CAT,1
1,TEMPSPACE1,/udata/ODS/IBMTBSP/db2inst1/NODE0000/ODS/T0000001/C0000000.TMP,1
1,USERSPACE1,/udata/ODS/IBMTBSP/db2inst1/NODE0000/ODS/T0000002/C0000000.LRG,1
1,TS_ODS_8KB_TMP,/udata/ODS/TEMP/db2inst1/NODE0000/ODS/T0000003/C0000000.TMP,1
1,TS_ODS_16KB_TMP,/udata/ODS/TEMP/db2inst1/NODE0000/ODS/T0000004/C0000000.TMP,1
1,TS_ODS_32KB_TMP,/udata/ODS/TEMP/db2inst1/NODE0000/ODS/T0000005/C0000000.TMP,1
1,TS_ODS_32KB_DATA,/udata/ODS/DATA/db2inst1/NODE0000/ODS/T0000006/C0000000.LRG,1
1,TS_ODS_16KB_DATA,/udata/ODS/DATA/db2inst1/NODE0000/ODS/T0000007/C0000000.LRG,1
1,TS_ODS_8KB_DATA,/udata/ODS/DATA/db2inst1/NODE0000/ODS/T0000008/C0000000.LRG,1
1,TS_ODS_32KB_DATA1,/udata/ODS/DATA/db2inst1/NODE0000/ODS/T0000009/C0000000.LRG,1


### <font color=green> Tables currently unavailable. Load pending. Reorg pending. Or need indexes rebuilt </font>

#### <font color=green> Check for inplace_reog_status=aborted that is not a problem </font>

In [11]:
%sql select distinct substr(tabschema, 1, 18) as schema, substr(tabname, 1,50) as tablename \
,available, reorg_pending, inplace_reorg_status, load_status, read_access_only, indexes_require_rebuild as indx_require_rebuild \
from  SYSIBMADM.ADMINTABINFO \
where available = 'N' or \
reorg_pending = 'Y' or \
inplace_reorg_status is not null or \
load_status is not null or \
read_access_only = 'Y' or \
indexes_require_rebuild = 'Y' \
order by 1,2 with ur;


 * db2+ibm_db://db2inst1:***@db2mbruat1:60000/ods
Done.


SCHEMA,tablename,available,reorg_pending,inplace_reorg_status,load_status,read_access_only,indx_require_rebuild
WMOSODS,ALLOC_INVN_DTL,Y,N,ABORTED,None,N,N
WMOSODS,ASN_DETAIL,Y,N,ABORTED,None,N,N
WMOSODS,ITEM_CBO,Y,N,ABORTED,None,N,N
WMOSODS,ITEM_FACILITY_MAPPING_WMS,Y,N,ABORTED,None,N,N
WMOSODS,ITEM_FACILITY_MAPPING_WMS_HISTORY,Y,N,ABORTED,None,N,N
WMOSODS,ITEM_SUPPLIER_XREF_CBO,Y,N,ABORTED,None,N,N
WMOSODS,ITEM_WMS,Y,N,ABORTED,None,N,N
WMOSODS,ITEM_WMS_HISTORY,Y,N,ABORTED,None,N,N
WMOSODS,LPN,Y,N,ABORTED,None,N,N
WMOSODS,LPN_DETAIL,Y,N,ABORTED,None,N,N
